In [ ]:
# --------------------------------
# Step 1: Imports
# --------------------------------
import os
import cv2
import csv
import json
import numpy as np
import matplotlib.pyplot as plt
from ultralytics import YOLO
from scipy.spatial.transform import Rotation as R
from plyfile import PlyData

# --------------------------------
# Step 2: Config
# --------------------------------
NUM_FRAMES = 240
MAX_DISTANCE = 7.0  # meters

PLY_DIR = "src/ply"
RGB_DIR = "src/rgb"
PREVIEW_DIR = "preview"
OUTPUT_DIR = "output"

os.makedirs(PREVIEW_DIR, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)

# --------------------------------
# Step 3: Load camera pose
# --------------------------------
with open("src/metadata.json") as f:
    pose = json.load(f)["poses"][0]

qx, qy, qz, qw = pose[:4]
tx, ty, tz = pose[4:7]

R_wc = R.from_quat([qx, qy, qz, qw]).as_matrix()
t_wc = np.array([tx, ty, tz])

# --------------------------------
# Step 4: Helpers
# --------------------------------
def load_ply_xyz(path):
    ply = PlyData.read(path)
    v = ply["vertex"]
    return np.vstack([v["x"], v["y"], v["z"]]).T

def project_points(points, fx, fy, cx, cy):
    Pc = (R_wc.T @ (points.T - t_wc[:, None])).T
    Z = Pc[:, 2]
    valid = Z > 0
    u = fx * (Pc[:, 0] / Z) + cx
    v = fy * (Pc[:, 1] / Z) + cy
    return np.stack([u, v], axis=1), Z, valid

def pixel_to_3d(u, v, points, fx, fy, cx, cy):
    uv, Z, valid = project_points(points, fx, fy, cx, cy)
    dist2 = (uv[:, 0] - u) ** 2 + (uv[:, 1] - v) ** 2
    idx = np.argmin(dist2)
    Pc = (R_wc.T @ (points[idx] - t_wc))
    return points[idx], np.linalg.norm(Pc)

# --------------------------------
# Step 5: YOLO
# --------------------------------
model = YOLO("yolov8n.pt")

# --------------------------------
# Step 6: Agent tracking storage
# --------------------------------
agent_positions = {}      # agent_id -> last_position
agent_records = []       # list of dicts per frame
next_agent_id = 0

def match_agent(point, max_dist=1.0):
    global next_agent_id
    for aid, last in agent_positions.items():
        if np.linalg.norm(last - point) < max_dist:
            agent_positions[aid] = point
            return aid
    aid = next_agent_id
    agent_positions[aid] = point
    next_agent_id += 1
    return aid

# --------------------------------
# Step 7: Main loop
# --------------------------------
for frame in range(NUM_FRAMES):
    print(f"Processing frame {frame}")

    ply_path = f"{PLY_DIR}/{frame:07d}.ply"
    rgb_path = f"{RGB_DIR}/rgb_{frame:03d}.jpg"

    if not os.path.exists(ply_path) or not os.path.exists(rgb_path):
        print("  Missing data, skipping")
        agent_records.append({})
        continue

    points = load_ply_xyz(ply_path)

    # Filter points beyond 7m
    Pc = (R_wc.T @ (points.T - t_wc[:, None])).T
    distances = np.linalg.norm(Pc, axis=1)
    points = points[distances <= MAX_DISTANCE]

    img = cv2.imread(rgb_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    h, w = img_rgb.shape[:2]

    fx = fy = 0.5 * w
    cx, cy = w / 2, h / 2

    results = model.predict(img_rgb, conf=0.3, classes=[0])
    frame_record = {}

    annotated = img_rgb.copy()

    for det in results[0].boxes:
        x1, y1, x2, y2 = det.xyxy[0].cpu().numpy()
        u_mid, v_mid = (x1 + x2) / 2, (y1 + y2) / 2

        point_3d, dist = pixel_to_3d(u_mid, v_mid, points, fx, fy, cx, cy)

        if dist > MAX_DISTANCE:
            continue

        agent_id = match_agent(point_3d)
        frame_record[agent_id] = point_3d

        # Draw
        cv2.rectangle(annotated, (int(x1), int(y1)), (int(x2), int(y2)), (255, 0, 0), 2)
        cv2.circle(annotated, (int(u_mid), int(v_mid)), 4, (0, 255, 0), -1)
        cv2.putText(
            annotated,
            f"ID {agent_id} | {dist:.2f}m",
            (int(x1), int(y1) - 5),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.5,
            (255, 255, 0),
            2
        )

    agent_records.append(frame_record)

    # Save preview
    out_path = f"{PREVIEW_DIR}/{frame}.jpg"
    cv2.imwrite(out_path, cv2.cvtColor(annotated, cv2.COLOR_RGB2BGR))

# --------------------------------
# Step 8: Export CSV
# --------------------------------
max_agents = max((max(r.keys()) if r else -1) for r in agent_records) + 1

csv_path = f"{OUTPUT_DIR}/agents_trajectory.csv"

with open(csv_path, "w", newline="") as f:
    writer = csv.writer(f)

    header = []
    for a in range(max_agents):
        header += [f"agent_{a}_x", f"agent_{a}_y", f"agent_{a}_z"]
    writer.writerow(header)

    for frame in agent_records:
        row = []
        for a in range(max_agents):
            if a in frame:
                row += frame[a].tolist()
            else:
                row += ["", "", ""]
        writer.writerow(row)

print("✅ Processing complete")
print(f"CSV saved to: {csv_path}")
print(f"Previews saved to: {PREVIEW_DIR}/")
